## Problem Set 1 - BDMLAE

Packages

In [8]:
import pandas as pd
import numpy as np
import pyreadr as pyr
import sklearn as sk
import scipy as sc
from scipy.stats import bootstrap
from sklearn.impute import KNNImputer

Cleaning

In [20]:
bd=pyr.read_r("C:/Users/ASUS/Documents/Universidad de los Andes/2022-2/Big Data and Machine Learning for Applied Economics/Taller 1/PS1_BD-ML/bdPS1.Rdata")
print(bd.keys())
df0=bd["df"]
df=df0.drop(df0[df0.age<18].index) #Restrict the data to legal age individuals (age>=18).
df=df.drop(df[df.ocu==0].index) #Restrict the data to employed individuals (ocu==1).
df=(df[["age", "clase", "college", "sizeFirm", "oficio", "hoursWorkUsual", "formal", "ingtot", "mes", "estrato1", "sex", 
"regSalud", "p6426", "p7090","p7510s7a1", "impa", "isa", "impaes", "isaes", "fex_c", "fweight"]]) #Relevant variables selection. 

odict_keys(['df'])


In [10]:
#imputer=KNNImputer(n_neighbors=10)
#imputer.fit(df)
#df=pd.DataFrame(imputer.transform(df))

In [21]:
oficio_d=pd.get_dummies(df["oficio"], prefix="oficio")
sizeFirm_d=pd.get_dummies(df["sizeFirm"], prefix="size")
estrato1_d=pd.get_dummies(df["estrato1"], prefix="estrato")
regSalud_d=pd.get_dummies(df["regSalud"], prefix="regs")
df=pd.merge(df, oficio_d, left_index=True, right_index=True)
df=pd.merge(df, sizeFirm_d, left_index=True, right_index=True)
df=pd.merge(df, estrato1_d, left_index=True, right_index=True)
df=pd.merge(df, regSalud_d, left_index=True, right_index=True)
df["college"]=np.where(df["college"]==2, 0, 1)
df["p7090"]=np.where(df["p7090"]==2, 0, 1)

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [29]:
ds=df[["age", "clase", "college", "hoursWorkUsual", "formal", "ingtot", "estrato_1", "estrato_2", "estrato_3", "estrato_4", "estrato_5", "estrato_6", "sex", "regs_1", "regs_2", "regs_3", "p7090", "p6426"]].describe(include="all")
ds=ds.T
ds=ds[["count", "mean", "std", "min", "50%", "max"]]
print(ds.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &    count &          mean &           std &   min &           50\% &           max \\
\midrule
age            &  16542.0 &  3.943616e+01 &  1.348284e+01 &  18.0 &  3.800000e+01 &  9.400000e+01 \\
clase          &  16542.0 &  1.000000e+00 &  0.000000e+00 &   1.0 &  1.000000e+00 &  1.000000e+00 \\
college        &  16542.0 &  1.000000e+00 &  0.000000e+00 &   1.0 &  1.000000e+00 &  1.000000e+00 \\
hoursWorkUsual &  16542.0 &           NaN &           NaN &   NaN &           NaN &           NaN \\
formal         &  16542.0 &           NaN &           NaN &   NaN &           NaN &           NaN \\
ingtot         &  16542.0 &  1.769379e+06 &  2.675628e+06 &   0.0 &  1.051160e+06 &  8.583333e+07 \\
estrato\_1      &  16542.0 &  1.071817e-01 &  3.093535e-01 &   0.0 &  0.000000e+00 &  1.000000e+00 \\
estrato\_2      &  16542.0 &  4.180873e-01 &  4.932596e-01 &   0.0 &  0.000000e+00 &  1.000000e+00 \\
estrato\_3      &  16542.0 &  3.616250e-01 &  4.804855e-0

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17864\2249692259.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ds.to_latex())


In [6]:
mis=df.isnull().sum()
mis=pd.DataFrame(mis)
ds=pd.merge(ds, mis, left_index=True, right_index=True)
ds

,count,mean,std,min,50%,max,0
age,16542.0,3.943616e+01,1.348284e+01,18.0,3.800000e+01,9.400000e+01,0
clase,16542.0,1.000000e+00,0.000000e+00,1.0,1.000000e+00,1.000000e+00,0
college,16542.0,3.194293e-01,4.662696e-01,0.0,0.000000e+00,1.000000e+00,0
hoursWorkUsual,16542.0,NaN,NaN,NaN,NaN,NaN,0
formal,16542.0,NaN,NaN,NaN,NaN,NaN,0
ingtot,16542.0,1.769379e+06,2.675628e+06,0.0,1.051160e+06,8.583333e+07,0
estrato_1,16542.0,1.071817e-01,3.093535e-01,0.0,0.000000e+00,1.000000e+00,0
estrato_2,16542.0,4.180873e-01,4.932596e-01,0.0,0.000000e+00,1.000000e+00,0
estrato_3,16542.0,3.616250e-01,4.804855e-01,0.0,0.000000e+00,1.000000e+00,0
estrato_4,16542.0,6.806916e-02,2.518722e-01,0.0,0.000000e+00,1.000000e+00,0
